In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [3]:
from implementations import *

In [4]:
# pre-processing: normalize each feature, then fill bad data with average (0)

num_feature = tX.shape[1]
col_avg = np.zeros(num_feature)
col_std = np.zeros(num_feature)

# calculate the average and stadard deviation of each data coloum
for i in range(num_feature):
    cur_col = tX[:, i]
    if i == num_feature-1:
        good_data = np.where(cur_col!=0)
    else:
        good_data = np.where(cur_col!=-999)
    col_avg[i] = np.mean(cur_col[good_data])
    col_std[i] = np.std(cur_col[good_data])
    
col_avg = col_avg[np.newaxis, :]
col_std = col_std[np.newaxis, :]

# pick bad data
bad_data = np.where(tX==-999)
bad_col_data = np.where(tX[:, -1]==0)

# normalization
tX = (tX-col_avg) / col_std

# filling bad data with average
tX[bad_data] = 0
tX[:, -1][bad_col_data] = 0

In [5]:
# required implementations

# max_iters = 50
# gamma = 0.1 # least square GD & logistic_regression & reg_logistic_regression
# gamma = 0.001 # least square SGD
# lambda_ = 0.1
# initial_w = np.zeros(tX.shape[1])

# weights, loss = least_squares_GD(y, tX, initial_w, max_iters, gamma)
# weights, loss = least_squares_SGD(y, tX, initial_w, max_iters, gamma)
# weights, loss = least_squares(y, tX)
# weights, loss = ridge_regression(y, tX, lambda_)
# weights, loss = logistic_regression(y, tX, initial_w, max_iters, gamma)
# weights, loss = reg_logistic_regression(y, tX, lambda_, initial_w, max_iters, gamma)

In [6]:
from my_helper import *

In [7]:
# polynomial data augmentation

degree = 10

initial_w = np.zeros(tX.shape[1]*degree)
x_poly = build_poly(tX, degree)

# ridge regression

K = 5
seed = 6

lambdas = np.logspace(-5, 2, 20)
for lambda_ in lambdas:
    
    # cross validation
    train_acc, val_acc = cross_validation(y, x_poly, K, seed, ridge_regression, lambda_)

    print("Lambda {labd}, Train accuracy {ta}, Validation Accuracy {va}".format(labd=lambda_, ta=np.mean(train_acc), va=np.mean(val_acc)))

Lambda 1e-05, Train accuracy 0.8161539999999998, Validation Accuracy 0.8157080000000001
Lambda 2.3357214690901213e-05, Train accuracy 0.816149, Validation Accuracy 0.8157080000000001
Lambda 5.4555947811685143e-05, Train accuracy 0.8161400000000001, Validation Accuracy 0.8157119999999999
Lambda 0.00012742749857031334, Train accuracy 0.816137, Validation Accuracy 0.8156960000000002
Lambda 0.00029763514416313193, Train accuracy 0.8161400000000001, Validation Accuracy 0.8157039999999999
Lambda 0.0006951927961775605, Train accuracy 0.8161390000000001, Validation Accuracy 0.8156880000000001
Lambda 0.001623776739188721, Train accuracy 0.8161400000000001, Validation Accuracy 0.8156880000000001
Lambda 0.00379269019073225, Train accuracy 0.816136, Validation Accuracy 0.815704
Lambda 0.008858667904100823, Train accuracy 0.81616, Validation Accuracy 0.81572
Lambda 0.02069138081114788, Train accuracy 0.816151, Validation Accuracy 0.815736
Lambda 0.04832930238571752, Train accuracy 0.816154, Validat

In [8]:
# best para of ridge regression

ratio = 0.9
degree = 10
lambda_ = 9.236708571873866e-05
seed = 6

x_poly = build_poly(tX, degree)
weights, loss = ridge_regression(y, x_poly, lambda_)

pred = predict_labels(weights, x_poly)
acc = cal_acc(y, pred)

print(acc)

0.816132


In [9]:
from my_implementations import *

In [10]:
# polynomial data augmentation

degree = 4

initial_w = np.zeros(tX.shape[1]*degree)
x_train, y_train, x_val, y_val = split_data(tX, y, ratio, seed)
x_train_poly = build_poly(x_train, degree)
x_val_poly = build_poly(x_val, degree)

# best para of logistic regression Newton

max_iters = 50
gamma = 0.4

weights, loss = logistic_regression_newton(y_train, x_train_poly, initial_w, max_iters, gamma)

y_train_pred = predict_labels(weights, x_train_poly)
train_acc = cal_acc(y_train, y_train_pred)
y_val_pred = predict_labels(weights, x_val_poly)
val_acc = cal_acc(y_val, y_val_pred)

print("Train accuracy {ta}, Validation Accuracy {va}".format(ta=train_acc, va=val_acc))

LS GD(0/49): loss=0.5637145770022863
LS GD(1/49): loss=0.5030965871924096
LS GD(2/49): loss=0.46773667027252425
LS GD(3/49): loss=0.44557017340963617
LS GD(4/49): loss=0.4314588286880356
LS GD(5/49): loss=0.42265411088294524
LS GD(6/49): loss=0.41754003926894434
LS GD(7/49): loss=0.41474659195900354
LS GD(8/49): loss=0.4133046495999577
LS GD(9/49): loss=0.4125344821257037
LS GD(10/49): loss=0.4121563852803343
LS GD(11/49): loss=0.41197343078576665


/home/xingyu/Desktop/EPFL_semester1/ml-project-1-ssy_mlproject1/project1/scripts/implementations.py:64: RuntimeWarning: overflow encountered in exp
  return 1 / (1+np.exp(-t))


LS GD(12/49): loss=0.4118857912121183
LS GD(13/49): loss=0.41184190408260984
LS GD(14/49): loss=0.41181806738449883
LS GD(15/49): loss=0.4118036026042454
LS GD(16/49): loss=0.41179434283349325
LS GD(17/49): loss=0.41178865543011756
LS GD(18/49): loss=0.4117852680502254
LS GD(19/49): loss=0.4117255400056201
LS GD(20/49): loss=0.4117181528356375
LS GD(21/49): loss=0.4117124056328053
LS GD(22/49): loss=0.41170793364373964
LS GD(23/49): loss=0.41170445427969854
LS GD(24/49): loss=0.41170174766943873
LS GD(25/49): loss=0.41169964251737484
LS GD(26/49): loss=0.4116980053593512
LS GD(27/49): loss=0.41169673224056175
LS GD(28/49): loss=0.41169574222483696
LS GD(29/49): loss=0.41169497232379115
LS GD(30/49): loss=0.4116943735357542
LS GD(31/49): loss=0.4116939077533708
LS GD(32/49): loss=0.41169354535077685
LS GD(33/49): loss=0.4116932633020138
LS GD(34/49): loss=0.4116930437144813
LS GD(35/49): loss=0.4116928726864973
LS GD(36/49): loss=0.41169273941788265
LS GD(37/49): loss=0.4116926355179444

## Generate predictions and save ouput in csv format for submission:

In [11]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

# pre-process
bad_data = np.where(tX_test==-999)
bad_col_data = np.where(tX_test[:, -1]==0)

tX_test = (tX_test-col_avg) / col_std
tX_test[bad_data] = 0
tX_test[:, -1][bad_col_data] = 0

In [12]:
OUTPUT_PATH = 'predictions.csv' # TODO: fill in desired name of output file for submission

# polynomial data augmentation
x_test_poly = build_poly(tX_test, degree)
y_pred = predict_labels(weights, x_test_poly)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)